In [ ]:
class LSTM_Malware(nn.Module):
    def __init__(self, embedding_dim, hidden_dim):
        super(LSTM_Malware, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(257, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, hidden_dim, 20, 10)
        self.maxpool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, 10, 5)
        self.maxpool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(hidden_dim, hidden_dim, 10, 5)
        self.maxpool3 = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=2, bidirectional=True, batch_first=True)
        self.hidden2label = nn.Linear(hidden_dim * 2, 1)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = x.transpose(1, 2)

        lstm_out, (h_n, c_n) = self.lstm(x)
        last_hidden = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        y = self.hidden2label(last_hidden)
        return y